# Projet numérique : câble sous-marin

## Questions théoriques

1) La loi des grands nombres permet, pour un nombre suffisamment important de simulations conditionnelles, d'estimer l'espérance conditionnelle comme la moyenne empirique de simulations conditionnelles.

2) On note U le vecteur des composantes de Z correspondant aux points de discrétisation sans observation, V celui corresppndant aux points de discrétisation avec observation. 
La loi conditionnelle de U sachant $V=v$ est définie par la densité conditionnelle suivante : 
$f_{U|V = v}(u) = \frac{1}{(2\pi)^{n/2}\sqrt{\det(CS_{U})}}\exp(-\frac{1}{2}(u - m_{U|V=v}){}^tCS_{U}^{-1}(u-m_{U|V=v})$.
En effet, le vecteur des profondeurs $Z$ est gaussien à densité et (au réordonnement des indices près) est tel que $Z = (Z_{1},\ldots,Z_{N})$ avec $U = (Z_1 ,\ldots,Z_{N-n+1}$ et $V = (Z_{N-n},\ldots,Z_N)$.
$CS_{U}$ désigne le complément de Schur du bloc $C_U$ de la matrice de covariance $C$ de $Z$ définie par blocs : $\begin{pmatrix} C_U&C_{U,V} \\ C_{V,U}&C_{V} \end{pmatrix}$ où $C_U = Cov(U,U)$, $C_Z = Cov(V,V)$ et $C_{U,V} = Cov(U,V)$.
On a donc $CS_U = C_U - C_{U,V}C_{V}^{-1}C_{V,U}$.
$m_{U|V=v} = m_U + C_{U,V}C_{V}^{-1}(v-m_V)$ est l'espérance conditionnelle de $U$ sachant $V$ ;  et CS_U est la covariance conditionnelle de $U$ sachant $V$.

3) Si $Y = (Y_1,\ldots,Y_p)$ est un vecteur de composantes gaussiennes indépendantes, toutes d'espérance nulle et de variance 1, alors le vecteur $Z=m+RY$ (où $R$ est une matrice $p\times p$ et $m$ est un vecteur de taille $p$) est gaussien d'espérance $m$ et de matrice de covariance $RR^{t}$.

4) Algorithme de simulation conditionnelle :
Calcul de la matrice de covariance du vecteur des profondeurs $Z$, grâce à la fonction $C$.
On extrait de cette matrice les blocs $C_U$, $C_{U,V}$, $C_{V,U}$,$C_V$ (on tient compte des indices des points d'observation).
Calcul de l'espérance conditionnelle $m$ de $U$ sachant $V$, grâce aux blocs extraits à l'étape précédente.
Calcul de la covariance conditionnelle $CS_U$. Comme la matrice de covariance $C$ du vecteur des profondeurs $Z$ est définie positive, $CS_U$ est aussi définie positive, on peut donc écrire sa décomposition de Cholesky $CS_U = RR^{t}$.
Simulation d'un vecteur gaussien $Y$ de $k$ variables indépendantes d'espérance nulle et de variance 1 (où $k$ est le nombre de points de discrétisation sans observation).
Calcul de $U = m + RY$.
On ainsi effectué une simulation conditionnelle de $U$ sachant $V = v$.

In [ ]:
hbjkbhbj